### 3- Helper for finding Transform & Timestamp needed from extrinsic calibration

In [ ]:
%pip install scipy plotly pandas nbformat

In [ ]:
!source /opt/ros/humble/setup.bash

### From Kalibr for camera to imu

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# cam0 to imu0 HD720 / 2 (640, 360) 
T_ic_cam0_HD720 = np.array([
    [-0.02139781, -0.04509792,  0.99875338,  0.09553478],
    [-0.9990368,  -0.0373147,  -0.0230888,   0.01664252],
    [ 0.03830944, -0.99828543, -0.04425603, -0.03314106],
    [ 0.,          0.,          0.,          1.        ]
])

# cam1 to imu0
T_ic_cam1_HD720 = np.array([
    [-0.02768216, -0.04476991,  0.99861372,  0.09508215],
    [-0.99887051, -0.03735585, -0.02936402, -0.04580398],
    [ 0.03861869, -0.99829865, -0.04368525, -0.03071331],
    [ 0.,          0.,          0.,          1.        ]
])

# csi_cam to imu0
T_ic_csi_cam = np.array([
    [-0.01067075,  0.64499989,  0.76410816,  0.05320717],
    [-0.99872944, -0.04451191,  0.02362622, -0.0016416 ],
    [ 0.04925082, -0.76288521,  0.64465535,  0.07154415],
    [ 0.,          0.,          0.,          1.        ]
])

# cam0 to imu0 VGA(672, 376) 
T_ic_cam0_vga = np.array([
    [ 0.01327902, -0.08917957,  0.99592704,  0.08108838],
    [-0.99990508,  0.00247407,  0.0135536,   0.03422853],
    [-0.0036727,  -0.99601249, -0.08913825, -0.03896725],
    [ 0.,          0.,          0.,          1.        ]
])

# cam1 to imu0
T_ic_cam1_vga = np.array([
    [ 0.01302763, -0.09208576,  0.99566585,  0.08183526],
    [-0.99990832,  0.00247717,  0.01331224, -0.02849814],
    [-0.0036923,  -0.995748,   -0.09204505, -0.03939789],
    [ 0.,          0.,          0.,          1.        ]
])

def extract_origin(tform):
    t = tform[:3, 3]
    r = R.from_matrix(tform[:3, :3])
    rpy = r.as_euler("xyz", degrees=False)
    return t, rpy

# cam0
xyz0, rpy0 = extract_origin(T_ic_cam0_HD720)
# cam1
xyz1, rpy1 = extract_origin(T_ic_cam1_HD720)

# Print with 5 decimal precision
print("----------------------------------------")
print("HD720 divided by 2 (640, 360) resolution")
print("----------------------------------------")
for name, xyz, rpy in [("cam0", xyz0, rpy0), ("cam1", xyz1, rpy1)]:
    print(f"{name} xyz: {xyz.round(5).tolist()}")
    print(f"{name} rpy: {rpy.round(5).tolist()}")

# cam0
xyz0, rpy0 = extract_origin(T_ic_cam0_vga)
# cam1
xyz1, rpy1 = extract_origin(T_ic_cam1_vga)
print("----------------------------------------")
print("VGA (672, 376) resolution")
print("----------------------------------------")
for name, xyz, rpy in [("cam0", xyz0, rpy0), ("cam1", xyz1, rpy1)]:
    print(f"{name} xyz: {xyz.round(5).tolist()}")
    print(f"{name} rpy: {rpy.round(5).tolist()}")

# CSI cam
xyz, rpy = extract_origin(T_ic_csi_cam)
print("----------------------------------------")
print("CSI cam")
print("----------------------------------------")
print(f"csi_cam xyz: {xyz.round(5).tolist()}")
print(f"csi_cam rpy: {rpy.round(5).tolist()}")

### Find xacro from camera intrinsic and extrinsic

In [ ]:
import numpy as np
np.float_ = np.float64 # NOTE: workaround for tf_transformations compatibility

from tf_transformations import quaternion_from_euler, euler_from_matrix, translation_matrix, quaternion_matrix, inverse_matrix, translation_from_matrix

kalibr_trans = [0.09553, 0.01664, -0.03314]
kalibr_rpy = [-1.6151, -0.03832, -1.59221] # Roll, Pitch, Yaw

# Convert Kalibr RPY to a quaternion, then a 4x4 matrix
kalibr_quat = quaternion_from_euler(kalibr_rpy[0], kalibr_rpy[1], kalibr_rpy[2])
T_optical_base = quaternion_matrix(kalibr_quat)
T_optical_base[0:3, 3] = kalibr_trans

camera_baseline = 0.06249531986870007
camera_height = 0.0265

# Transform from camera_link to camera_center
T_center_camlink = translation_matrix([0, 0, camera_height / 2.0])

# Transform from camera_center to left_camera_frame
T_left_center = translation_matrix([0, camera_baseline / 2.0, 0])

# Transform from left_camera_frame to left_camera_optical_frame
optical_rpy = [-np.pi/2, 0, -np.pi/2]
optical_quat = quaternion_from_euler(optical_rpy[0], optical_rpy[1], optical_rpy[2])
T_optical_left = quaternion_matrix(optical_quat)

# Get the full transform from camera_link to optical_frame
# T_optical^cam_link = T_center^cam_link * T_left^center * T_optical^left
T_optical_camlink = np.dot(T_center_camlink, np.dot(T_left_center, T_optical_left))

# T_cam_link^base = T_optical^base * inv(T_optical^cam_link)
T_camlink_base = np.dot(T_optical_base, inverse_matrix(T_optical_camlink))

# Extract Final XYZ and RPY ---
final_xyz = translation_from_matrix(T_camlink_base)
final_rpy = euler_from_matrix(T_camlink_base, 'sxyz') # 'sxyz' for roll, pitch, yaw

print("--- Final Transform for URDF (base_link -> camera_link) ---")
print(f"XYZ (m): [{final_xyz[0]:.6f}, {final_xyz[1]:.6f}, {final_xyz[2]:.6f}]")
print(f"RPY (rad): [{final_rpy[0]:.6f}, {final_rpy[1]:.6f}, {final_rpy[2]:.6f}]")

print(f'<origin xyz="{final_xyz[0]:.6f} {final_xyz[1]:.6f} {final_xyz[2]:.6f}" rpy="{final_rpy[0]:.6f} {final_rpy[1]:.6f} {final_rpy[2]:.6f}"/>')

In [ ]:
import numpy as np
import plotly.graph_objects as go
from transforms3d.euler import euler2quat, quat2euler
from transforms3d.quaternions import quat2mat
from transforms3d.affines import compose

def plot_frame_plotly(fig, T, label, length=0.05, line_width=4):
    """
    Plots a 3D coordinate frame on a Plotly figure.
    :param fig: Plotly figure object
    :param T: 4x4 transformation matrix (numpy array)
    :param label: Name of the frame
    :param length: Length of the axis arrows
    :param line_width: Width of the axis lines
    """
    origin = T[0:3, 3]
    R = T[0:3, 0:3]

    # Define axis vectors
    x_axis = R[:, 0] * length
    y_axis = R[:, 1] * length
    z_axis = R[:, 2] * length

    # Add X-axis (red)
    fig.add_trace(go.Scatter3d(
        x=[origin[0], origin[0] + x_axis[0]],
        y=[origin[1], origin[1] + x_axis[1]],
        z=[origin[2], origin[2] + x_axis[2]],
        mode='lines',
        line=dict(color='red', width=line_width),
        name=f'{label} X-axis',
        showlegend=False
    ))

    # Add Y-axis (green)
    fig.add_trace(go.Scatter3d(
        x=[origin[0], origin[0] + y_axis[0]],
        y=[origin[1], origin[1] + y_axis[1]],
        z=[origin[2], origin[2] + y_axis[2]],
        mode='lines',
        line=dict(color='green', width=line_width),
        name=f'{label} Y-axis',
        showlegend=False
    ))

    # Add Z-axis (blue)
    fig.add_trace(go.Scatter3d(
        x=[origin[0], origin[0] + z_axis[0]],
        y=[origin[1], origin[1] + z_axis[1]],
        z=[origin[2], origin[2] + z_axis[2]],
        mode='lines',
        line=dict(color='blue', width=line_width),
        name=f'{label} Z-axis',
        showlegend=False
    ))

    # Add label text
    fig.add_trace(go.Scatter3d(
        x=[origin[0]], y=[origin[1]], z=[origin[2]],
        mode='text',
        text=[label],
        textfont=dict(color='black', size=12),
        textposition="top center",
        name=label,
        showlegend=True
    ))

# Helper function to convert rpy to quaternion using transforms3d (quaternion_from_euler equivalent)
def quaternion_from_euler(roll, pitch, yaw):
    return euler2quat(roll, pitch, yaw, axes='sxyz') # static XYZ (intrinsic) or dynamic ZYX (extrinsic)

# Helper function to create a translation matrix (translation_matrix equivalent)
def translation_matrix(translation):
    M = np.identity(4)
    M[0:3, 3] = translation
    return M

# Helper function to create a quaternion matrix (quaternion_matrix equivalent)
def quaternion_matrix(quat):
    # transforms3d.quaternions.quat2mat returns a 3x3 rotation matrix
    R = quat2mat(quat)
    M = np.identity(4)
    M[0:3, 0:3] = R
    return M

# Helper function to create transformation matrix from quaternion and translation
def transform_from_quat_trans(quat, trans):
    """
    Create 4x4 transformation matrix from quaternion and translation
    :param quat: quaternion [qx, qy, qz, qw]
    :param trans: translation [tx, ty, tz]
    """
    # Convert quaternion [qx, qy, qz, qw] to [qw, qx, qy, qz] format for transforms3d
    quat_transforms3d = [quat[3], quat[0], quat[1], quat[2]]  # [qw, qx, qy, qz]
    T = quaternion_matrix(quat_transforms3d)
    T[0:3, 3] = trans
    return T

# base_link is the origin (px4_imu)
T_base_link = np.identity(4)

# T_camlink^base
T_camlink_trans = [0.094264, -0.015072, -0.045170]
T_camlink_rpy = [-0.038358, 0.044271, -0.023112]
T_camlink_quat = quaternion_from_euler(*T_camlink_rpy)
T_camlink_base = quaternion_matrix(T_camlink_quat)
T_camlink_base[0:3, 3] = T_camlink_trans

# T_left_optical^base (from Kalibr)
T_left_trans = [0.09553, 0.01664, -0.03314]
T_left_rpy = [-1.6151, -0.03832, -1.59221]
T_left_quat = quaternion_from_euler(*T_left_rpy)
T_left_optical_base = quaternion_matrix(T_left_quat)
T_left_optical_base[0:3, 3] = T_left_trans

# T_right_optical^base (from Kalibr)
T_right_trans = [0.09508, -0.0458, -0.03071]
T_right_rpy = [-1.61453, -0.03863, -1.5985]
T_right_quat = quaternion_from_euler(*T_right_rpy)
T_right_optical_base = quaternion_matrix(T_right_quat)
T_right_optical_base[0:3, 3] = T_right_trans

# T_front_up_optical^base (from Kalibr)
T_front_up_trans = [0.05321, -0.00164, 0.07154]
T_front_up_rpy = [-0.8692, -0.04927, -1.58148]
T_front_up_quat = quaternion_from_euler(*T_front_up_rpy)
T_front_up_optical_base = quaternion_matrix(T_front_up_quat)
T_front_up_optical_base[0:3, 3] = T_front_up_trans


# T_lidar^left_cam (LiDAR to left camera transform)
T_lidar_left_quat = [0.51857204, -0.51216204, 0.48526003, 0.48300703]  # [qx, qy, qz, qw]
T_lidar_left_trans = [0.01281634, -0.12085161, -0.08237267]  # [tx, ty, tz]

T_lidar_left_cam = transform_from_quat_trans(T_lidar_left_quat, T_lidar_left_trans)

# Calculate T_lidar^base = T_left_optical^base * T_lidar^left_cam
# NOTE: This assumes lidar->left_cam transform is relative to the left_cam_optical frame
T_lidar_base = T_left_optical_base @ T_lidar_left_cam
# Extract LiDAR to base_link RPY
T_lidar_base_rotation = T_lidar_base[0:3, 0:3]
# Convert rotation matrix to quaternion first, then to RPY
from scipy.spatial.transform import Rotation as R
r = R.from_matrix(T_lidar_base_rotation)
T_lidar_base_quat = r.as_quat()  # [qx, qy, qz, qw]
T_lidar_base_rpy = r.as_euler('xyz', degrees=False)  # [roll, pitch, yaw] in radians

# Print transform
print("LiDAR to Left Camera Transform:")
print("Translation:", T_lidar_left_trans)
print("Quaternion [qx, qy, qz, qw]:", T_lidar_left_quat)
print("\nLiDAR to base_link Transform:")
print("Translation [tx, ty, tz]:", T_lidar_base[0:3, 3])
print("Quaternion [qx, qy, qz, qw]:", T_lidar_base_quat)
print("RPY [roll, pitch, yaw] (radians):", T_lidar_base_rpy)

# Plot
fig = go.Figure()

# Plot each frame
plot_frame_plotly(fig, T_base_link, 'base_link')
plot_frame_plotly(fig, T_camlink_base, 'camera_link')
plot_frame_plotly(fig, T_left_optical_base, 'left_cam_optical')
plot_frame_plotly(fig, T_right_optical_base, 'right_cam_optical')
plot_frame_plotly(fig, T_front_up_optical_base, 'front_up_cam_optical')
plot_frame_plotly(fig, T_lidar_base, 'laser', length=0.08, line_width=3)

# Update layout for 3D plot
fig.update_layout(
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data', # Ensures equal aspect ratio
    ),
    title='Visualization of Camera and LiDAR TF Frames',
    showlegend=True,
    scene_camera=dict(
        up=dict(x=0, y=0, z=1),  # Z-axis points upwards
        center=dict(x=0, y=0, z=0), # Center of the view
        eye=dict(x=1.5, y=1.5, z=1.5) # Camera position
    )
)

fig.show()

### For downwards rangesensor

- Record data while stationary at different orientations:
```
Level (0°, 0°)
Roll: -15°, -10°, -5°, +5°, +10°, +15°
Pitch: -15°, -10°, -5°, +5°, +10°, +15°

Hold each pose for 5-10 seconds to get stable readings
```

- Keep vehicle level, vary height:
```
0.5m, 1.0m, 1.5m, 2.0m above ground
Hold each height for 5-10 seconds
```

- Dynamic movement:
```
Rectangular hover pattern with random attitude changes
```

In [ ]:
%pip install transforms3d

In [ ]:
import rosbag2_py
import numpy as np
from scipy.optimize import minimize
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from rclpy.serialization import deserialize_message
from sensor_msgs.msg import Imu
from sensor_msgs.msg import Range

class RangeIMUCalibration:
    def __init__(self, bag_path, imu_topic, range_topic):
        self.bag_path = bag_path
        self.imu_topic = imu_topic
        self.range_topic = range_topic
        
        # Store synchronized data
        self.imu_data = []
        self.range_data = []
        self.timestamps = []
        
        # Calibration parameters [tx, ty, tz, rx, ry, rz]
        self.initial_guess = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        
    def read_bag(self):
        """Read ROS2 bag and extract synchronized IMU and range data"""
        reader = rosbag2_py.SequentialReader()
        storage_options = rosbag2_py.StorageOptions(uri=self.bag_path, storage_id='sqlite3')
        converter_options = rosbag2_py.ConverterOptions('', '')
        reader.open(storage_options, converter_options)
        
        imu_msgs = {}
        range_msgs = {}
        
        while reader.has_next():
            (topic, data, timestamp) = reader.read_next()
            
            if topic == self.imu_topic:
                msg = deserialize_message(data, Imu)
                imu_msgs[timestamp] = msg
                
            elif topic == self.range_topic:
                msg = deserialize_message(data, Range)
                range_msgs[timestamp] = msg
        
        # Synchronize messages by timestamp
        self.synchronize_messages(imu_msgs, range_msgs)
        print(f"Synchronized {len(self.timestamps)} message pairs")
        
    def synchronize_messages(self, imu_msgs, range_msgs, tolerance=10e6):  # 10ms in nanoseconds
        """Synchronize IMU and range messages by timestamp"""
        imu_times = sorted(imu_msgs.keys())
        range_times = sorted(range_msgs.keys())
        
        for imu_time in imu_times:
            # Find closest range message
            closest_range_time = min(range_times, key=lambda x: abs(x - imu_time))
            
            if abs(imu_time - closest_range_time) <= tolerance:
                # Extract IMU data
                imu_msg = imu_msgs[imu_time]
                orientation = [imu_msg.orientation.x, imu_msg.orientation.y, 
                             imu_msg.orientation.z, imu_msg.orientation.w]
                
                # Extract range data
                range_msg = range_msgs[closest_range_time]
                range_value = range_msg.range
                
                # Store valid measurements
                if range_msg.min_range <= range_value <= range_msg.max_range:
                    self.imu_data.append(orientation)
                    self.range_data.append(range_value)
                    self.timestamps.append(imu_time)
                    
    def transform_point(self, params):
        """Apply transformation to range sensor measurements"""
        tx, ty, tz, rx, ry, rz = params
        
        # Translation vector
        translation = np.array([tx, ty, tz])
        
        # Rotation matrix from Euler angles (roll, pitch, yaw)
        rotation_matrix = R.from_euler('xyz', [rx, ry, rz]).as_matrix()
        
        transformed_ranges = []
        
        for i, (quat, measured_range) in enumerate(zip(self.imu_data, self.range_data)):
            # Convert quaternion to rotation matrix (IMU orientation)
            imu_rotation = R.from_quat(quat).as_matrix()
            
            # Range sensor measurement in sensor frame (assume pointing down -Z)
            range_vector_sensor = np.array([0, 0, -measured_range])
            
            # Transform to IMU frame
            range_vector_imu = rotation_matrix @ range_vector_sensor + translation
            
            # Transform to world frame using IMU orientation
            range_vector_world = imu_rotation @ range_vector_imu
            
            # Expected ground height (assuming flat surface at z=0)
            # The Z component should be consistent for flat ground
            transformed_ranges.append(range_vector_world[2])
            
        return np.array(transformed_ranges)
    
    def cost_function(self, params):
        """Cost function for calibration optimization"""
        transformed_ranges = self.transform_point(params)
        
        # For flat surface, all Z values should be similar
        # Cost is variance of ground height estimates
        cost = np.var(transformed_ranges)
        
        # Add regularization to prevent extreme parameters
        reg_weight = 0.01
        cost += reg_weight * np.sum(params**2)
        
        return cost
    
    def calibrate(self):
        """Perform extrinsic calibration optimization"""
        print("Starting calibration optimization...")
        
        # Bounds for parameters
        bounds = [
            (-0.15, 0.15),   # tx
            (-0.15, 0.15),   # ty
            (-0.15, 0.15),   # tz
            (-np.pi/6, np.pi/6),  # rx
            (-np.pi/6, np.pi/6),  # ry
            (-np.pi/6, np.pi/6),  # rz
        ]
        
        # Run optimization
        result = minimize(
            self.cost_function,
            self.initial_guess,
            method='L-BFGS-B',
            bounds=bounds,
            options={'maxiter': 1000}
        )
        
        if result.success:
            print("Calibration successful!")
            tx, ty, tz, rx, ry, rz = result.x
            
            print(f"\nExtrinsic calibration parameters:")
            print(f"Translation (m): [{tx:.4f}, {ty:.4f}, {tz:.4f}]")
            print(f"Rotation (rad): [{rx:.4f}, {ry:.4f}, {rz:.4f}]")
            print(f"Rotation (deg): [{np.degrees(rx):.2f}, {np.degrees(ry):.2f}, {np.degrees(rz):.2f}]")
            print(f"Final cost: {result.fun:.6f}")
            
            # Generate ROS2 static transform publisher command
            self.generate_tf_static_command(tx, ty, tz, rx, ry, rz)
            
            return result.x
        else:
            print("Calibration failed!")
            print(result.message)
            return None
    
    def generate_tf_static_command(self, tx, ty, tz, rx, ry, rz):
        """Generate ROS2 static transform publisher command"""
        # Convert Euler angles to quaternion
        quat = R.from_euler('xyz', [rx, ry, rz]).as_quat()
        
        print(f"\nROS2 static transform publisher command:")
        print(f"ros2 run tf2_ros static_transform_publisher "
              f"{tx:.4f} {ty:.4f} {tz:.4f} "
              f"{quat[0]:.4f} {quat[1]:.4f} {quat[2]:.4f} {quat[3]:.4f} "
              f"base_link range_sensor")
              
        print(f"\nURDF/SDF format:")
        print(f"<origin xyz='{tx:.4f} {ty:.4f} {tz:.4f}' "
              f"rpy='{rx:.4f} {ry:.4f} {rz:.4f}' />")
    
    def validate_calibration(self, params):
        """Validate calibration results"""
        transformed_ranges = self.transform_point(params)
        
        print(f"\nValidation results:")
        print(f"Ground height std dev: {np.std(transformed_ranges):.4f} m")
        print(f"Ground height mean: {np.mean(transformed_ranges):.4f} m")
        print(f"Ground height range: [{np.min(transformed_ranges):.4f}, {np.max(transformed_ranges):.4f}] m")
        
        # Plot results
        plt.figure(figsize=(12, 8))
        
        plt.subplot(2, 2, 1)
        plt.plot(transformed_ranges)
        plt.title('Estimated Ground Height')
        plt.ylabel('Height (m)')
        plt.xlabel('Sample')
        
        plt.subplot(2, 2, 2)
        plt.hist(transformed_ranges, bins=30)
        plt.title('Ground Height Distribution')
        plt.xlabel('Height (m)')
        plt.ylabel('Count')
        
        plt.subplot(2, 2, 3)
        plt.plot(self.range_data)
        plt.title('Raw Range Measurements')
        plt.ylabel('Range (m)')
        plt.xlabel('Sample')
        
        plt.subplot(2, 2, 4)
        # Plot IMU orientations (roll, pitch)
        orientations = [R.from_quat(q).as_euler('xyz') for q in self.imu_data]
        orientations = np.array(orientations)
        plt.plot(np.degrees(orientations[:, 0]), label='Roll')
        plt.plot(np.degrees(orientations[:, 1]), label='Pitch')
        plt.title('IMU Orientation')
        plt.ylabel('Angle (deg)')
        plt.xlabel('Sample')
        plt.legend()
        
        plt.tight_layout()
        plt.show()

def main():
    # Paths
    bag_path = "../bags/calib_range1"
    imu_topic = "/mavros/imu/data"
    range_topic = "/mavros/hps167_pub"
    
    # Initialize calibration
    calibrator = RangeIMUCalibration(bag_path, imu_topic, range_topic)
    
    # Read bag data
    calibrator.read_bag()
    
    if len(calibrator.timestamps) < 100:
        print("Warning: Limited data points. Consider collecting more data.")
    
    # Perform calibration
    result = calibrator.calibrate()
    
    if result is not None:
        # Validate results
        calibrator.validate_calibration(result)

if __name__ == "__main__":
    main()

### 2D Lidar Scan to Zed SDK PoinCloud

In [ ]:
%pip install open3d opencv-python==4.10.0.82

In [ ]:
#!/usr/bin/env python3
"""
LiDAR-Camera-IMU Extrinsic Calibration using Open3D optimization
Outputs URDF parameters for rplidar joint
"""

import numpy as np
import open3d as o3d
import rosbag2_py
from rclpy.serialization import deserialize_message
from sensor_msgs.msg import LaserScan, PointCloud2, Image, Imu
import sensor_msgs_py.point_cloud2 as pc2
from geometry_msgs.msg import TransformStamped
import cv2
from cv_bridge import CvBridge
from scipy.optimize import minimize
import math
from collections import defaultdict

class LiDARCalibratorOptimizer:
    def __init__(self, bag_path):
        self.bag_path = bag_path
        self.bridge = CvBridge()
        
        # Kalibr calibration results
        self.T_cam0_imu = np.array([
            [ 0.05518993, -0.095478, 0.99390041, 0.09185485],
            [-0.9982956, 0.0136382, 0.05674413, 0.03547225],
            [-0.01897283, -0.99533811, -0.09456258, -0.03660268],
            [ 0., 0., 0., 1.]
        ])
        
        # Data storage
        self.laser_scans = []
        self.point_clouds = []
        self.imu_data = []
        self.timestamps = defaultdict(list)
        
        # Initial guess for LiDAR pose (modify based on your mounting)
        self.initial_guess = [0.0, 0.0, 0.1, 0.0, 0.0, 0.0]  # [x, y, z, roll, pitch, yaw]
        
    def load_bag_data(self):
        """Load synchronized data from ROS2 bag"""
        storage_options = rosbag2_py.StorageOptions(uri=self.bag_path, storage_id='sqlite3')
        converter_options = rosbag2_py.ConverterOptions(
            input_serialization_format='cdr',
            output_serialization_format='cdr'
        )
        
        reader = rosbag2_py.SequentialReader()
        reader.open(storage_options, converter_options)
        
        topic_types = reader.get_all_topics_and_types()
        type_map = {topic.name: topic.type for topic in topic_types}
        
        while reader.has_next():
            topic, data, timestamp = reader.read_next()
            
            if topic == '/scan':
                msg = deserialize_message(data, LaserScan)
                self.laser_scans.append((timestamp, msg))
                self.timestamps['laser'].append(timestamp)
                
            elif topic == '/zed_node/point_cloud/cloud_registered':
                msg = deserialize_message(data, PointCloud2)
                self.point_clouds.append((timestamp, msg))
                self.timestamps['pointcloud'].append(timestamp)
                
            elif topic == '/mavros/imu/data':
                msg = deserialize_message(data, Imu)
                self.imu_data.append((timestamp, msg))
                self.timestamps['imu'].append(timestamp)
        
        print(f"Loaded: {len(self.laser_scans)} laser, {len(self.point_clouds)} pointcloud, {len(self.imu_data)} IMU")
    
    def synchronize_data(self, time_tolerance=5e7):  # 50ms tolerance
        """Find temporally synchronized sensor data"""
        synced_data = []
        time_offsets = []
        
        for pc_ts, pc_msg in self.point_clouds:
            # Find closest laser scan
            laser_diffs = [abs(ls_ts - pc_ts) for ls_ts, _ in self.laser_scans]
            if min(laser_diffs) < time_tolerance:
                laser_idx = laser_diffs.index(min(laser_diffs))
                laser_ts, laser_msg = self.laser_scans[laser_idx]
                
                # Find closest IMU
                imu_diffs = [abs(imu_ts - pc_ts) for imu_ts, _ in self.imu_data]
                if min(imu_diffs) < time_tolerance:
                    imu_idx = imu_diffs.index(min(imu_diffs))
                    imu_ts, imu_msg = self.imu_data[imu_idx]
                    
                    time_offset = (laser_ts - pc_ts) * 1e-9  # Convert to seconds
                    time_offsets.append(time_offset)
                    
                    synced_data.append({
                        'timestamp': pc_ts,
                        'pointcloud': pc_msg,
                        'laser': laser_msg,
                        'imu': imu_msg,
                        'time_diffs': {'laser': laser_ts - pc_ts, 'imu': imu_ts - pc_ts}
                    })
        
        if time_offsets:
            avg_offset = np.mean(time_offsets)
            std_offset = np.std(time_offsets)
            print(f"Average time offset (laser - pointcloud): {avg_offset:.6f} ± {std_offset:.6f} seconds")
            print(f"Found {len(synced_data)} synchronized measurements")
        
        return synced_data
    
    def laser_to_points(self, laser_msg, z_height=None):
        """Convert LaserScan to 3D points"""
        # Use initial guess z offset if z_height not provided
        if z_height is None:
            z_height = self.initial_guess[2]  # Use initial z guess
        
        angles = np.arange(laser_msg.angle_min, laser_msg.angle_max + laser_msg.angle_increment, 
                          laser_msg.angle_increment)
        ranges = np.array(laser_msg.ranges)
        
        # Filter valid ranges
        valid_idx = (ranges >= laser_msg.range_min) & (ranges <= laser_msg.range_max) & np.isfinite(ranges)
        angles = angles[valid_idx]
        ranges = ranges[valid_idx]
        
        # Convert to cartesian
        x = ranges * np.cos(angles)
        y = ranges * np.sin(angles)
        z = np.full_like(x, z_height)
        
        return np.column_stack((x, y, z))
    
    def pointcloud_to_numpy(self, pc_msg):
        """Convert ROS PointCloud2 to numpy array"""
        points = []
        for point in pc2.read_points(pc_msg, field_names=("x", "y", "z"), skip_nans=True):
            points.append([point[0], point[1], point[2]])
        return np.array(points)
    
    def transform_points(self, points, transform_params):
        """Apply transformation to points"""
        x, y, z, roll, pitch, yaw = transform_params
        
        # Create transformation matrix
        R_x = np.array([[1, 0, 0], [0, np.cos(roll), -np.sin(roll)], [0, np.sin(roll), np.cos(roll)]])
        R_y = np.array([[np.cos(pitch), 0, np.sin(pitch)], [0, 1, 0], [-np.sin(pitch), 0, np.cos(pitch)]])
        R_z = np.array([[np.cos(yaw), -np.sin(yaw), 0], [np.sin(yaw), np.cos(yaw), 0], [0, 0, 1]])
        
        R = R_z @ R_y @ R_x
        t = np.array([x, y, z])
        
        return (R @ points.T).T + t
    
    def compute_icp_cost(self, laser_points, pc_points):
        """Compute ICP-style alignment cost"""
        if len(laser_points) == 0 or len(pc_points) == 0:
            return float('inf')
        
        # Create Open3D point clouds
        laser_pcd = o3d.geometry.PointCloud()
        laser_pcd.points = o3d.utility.Vector3dVector(laser_points)
        
        pc_pcd = o3d.geometry.PointCloud()
        pc_pcd.points = o3d.utility.Vector3dVector(pc_points)
        
        # Downsample for efficiency
        if len(pc_points) > 5000:
            pc_pcd = pc_pcd.voxel_down_sample(0.05)
        
        # Compute nearest neighbor distances
        distances = laser_pcd.compute_point_cloud_distance(pc_pcd)
        return np.mean(distances)
    
    def project_pointcloud_to_2d(self, points, z_range=(-0.2, 0.2)):
        """Project 3D pointcloud to 2D laser scan plane"""
        # Filter points near laser height
        mask = (points[:, 2] >= z_range[0]) & (points[:, 2] <= z_range[1])
        if np.sum(mask) == 0:
            return np.array([]).reshape(0, 3)
        
        filtered_points = points[mask]
        # Set Z to laser plane
        filtered_points[:, 2] = 0.0
        return filtered_points
    
    def objective_function(self, params, synced_data):
        """Optimization objective function"""
        total_cost = 0.0
        valid_pairs = 0
        
        for data in synced_data[:min(50, len(synced_data))]:  # Limit for speed
            # Transform laser points to camera frame
            laser_points = self.laser_to_points(data['laser'])
            if len(laser_points) == 0:
                continue
                
            # Apply current transformation guess
            transformed_laser = self.transform_points(laser_points, params)
            
            # Transform to camera frame (laser -> imu -> camera)
            laser_in_camera = (self.T_cam0_imu @ np.hstack([transformed_laser, np.ones((len(transformed_laser), 1))]).T).T[:, :3]
            
            # Get pointcloud data
            pc_points = self.pointcloud_to_numpy(data['pointcloud'])
            if len(pc_points) == 0:
                continue
            
            # Project pointcloud to laser plane for comparison
            pc_2d = self.project_pointcloud_to_2d(pc_points)
            if len(pc_2d) == 0:
                continue
            
            # Compute alignment cost
            cost = self.compute_icp_cost(laser_in_camera, pc_2d)
            if not np.isinf(cost):
                total_cost += cost
                valid_pairs += 1
        
        if valid_pairs == 0:
            return float('inf')
        
        return total_cost / valid_pairs
    
    def optimize_extrinsics(self, synced_data):
        """Optimize LiDAR extrinsic parameters"""
        print("Starting optimization...")
        
        # Bounds for optimization (reasonable physical constraints)
        bounds = [
            (-0.25, 0.25),   # x translation
            (-0.25, 0.25),   # y translation
            (-0.25, 0.25),   # z translation
            (-np.pi/6, np.pi/6),  # roll
            (-np.pi/6, np.pi/6),  # pitch
            (-np.pi/6, np.pi/6)       # yaw
        ]
        
        result = minimize(
            self.objective_function,
            self.initial_guess,
            args=(synced_data,),
            bounds=bounds,
            method='L-BFGS-B',
            options={'maxiter': 100, 'disp': True}
        )
        
        return result
    
    def run_calibration(self):
        """Main calibration pipeline"""
        print("Loading bag data...")
        self.load_bag_data()
        
        print("Synchronizing sensor data...")
        synced_data = self.synchronize_data()
        
        if len(synced_data) < 10:
            print("Warning: Very few synchronized measurements found!")
            return None
        
        print("Running optimization...")
        result = self.optimize_extrinsics(synced_data)
        
        if result.success:
            print(f"\nOptimization converged!")
            print(f"Final cost: {result.fun:.6f}")
            print(f"Parameters: {result.x}")
            
            return result.x
        else:
            print(f"Optimization failed: {result.message}")
            return None


def main():
    # Usage
    bag_path = "../bags/calib_zed_720_lidar"
    
    calibrator = LiDARCalibratorOptimizer(bag_path)
    
    # Set initial guess based on sensor mounting
    calibrator.initial_guess = [0.0, 0.03, 0.07, 0.0, 0.0, 0.0]  # [x, y, z, roll, pitch, yaw]
    
    result = calibrator.run_calibration()
    
    if result is not None:
        print("Calibration completed successfully!")
        print("Check 'lidar_extrinsics.urdf.xacro' for URDF parameters")
    else:
        print("Calibration failed!")


if __name__ == "__main__":
    main()